# Sequence Labelling

In this session we will build an HMM model for PoS-tagging and then CRF and neural models for Named Entity Recognition.

## Building a simple Hidden Markov Model

In this first part of the lab we will build a very simple bigram HMM using probability estimates over the Brown corpus, which is Part-of-Speech tagged.

Recall from course 6: probability estimates (with MLE estimation) can be calculated by dividing the number of occurrences of a bigram by the number of occurrences of the first word.

First of all, we import the corpus where we will estimate the probabilities:


In [109]:
from nltk.corpus import brown

This corpus is in the form of sequences of sentences, where each sentence is made by a sequence of pairs (word, POS-tag), like this:

In [110]:
brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

We recall (from the course) that a Hidden Markov Model is composed by:

- A set of $N$ states $Q = \{q_1, q_2, \ldots, q_N\}$
- A transition probability matrix $A=a_{11}\ldots a_{ij} \ldots a_{NN}$, where each $a_{ij}$ represents the probability of transitioning from state $q_i$ to $q_j$; note that $\sum_{j=1}^N{a_{ij}} = 1 \forall i$
- A sequence of $T$ observations $O = o_1, o_2, \ldots o_T$, each one drawn from a vocabulary of size $V=v_1, v_2, \ldots, v_M$, of size $M$;
- A sequence of *observation likelihoods* $B=b_i(o_t)$, also called **emission probabilities**, each expressing the probability of an observation $o_t$ being generated from a state $q_i$;
- Finally, an initial probability distribution $\Pi = \pi_i, \pi_2, \ldots, \pi_N$ where $\pi_i$ indicates the probability that the Markov chain will start from state $q_i$. Some states $q_j$ may have $\pi_j = 0$, meaning that they cannot be initial states. Also, $\sum_{i=1}^N{\pi_i}=1$.

In our case, the set of states $Q$ is made by the vocabulary of labels (the POS-tags). The vocabulary $V$ corresponds to the word vocabulary (i.e. all the set of different words that appear in our corpus). The observations correspond to the sentences in the corpus.

We will now split our corpus in a training and test set:



In [111]:
corpus = brown.tagged_sents()

training = corpus[:-10]
testing = corpus[-10:]

**Exercise 1**: Extract $Q$ and $V$ from the Brown corpus and determine their respective size

In [112]:
#insert your solution here
# set of states Q and the vocabulary V
Q = {}
V = {}
i = 0
j = 0
for sentence in corpus:
    for word, tag in sentence:
        if tag not in Q:
            Q[tag] = i
            i += 1
        if word not in V:
            V[word] = j
            j += 1
        
print("Number of states in Q: ", len(Q))
print("Number of words in V: ", len(V))

Number of states in Q:  472
Number of words in V:  56057


**Exercise 2**: Create the matrices ($A$, $B$ and $\Pi$) by using the probabilities estimated on the training set; since we are considering bigrams, the probabilities of the transition matrix will be calculated as $\frac{count(t_{-1}, t)}{count(t_{-1})}$.

*Important*: you will need to add smoothing (for instance Lidstone with $k=0.1$) on $B$ otherwise the output will be $0$.

In [113]:
#insert your solution here
import numpy as np

# initialize pi matrix, A matrix and B matrix
pi = np.zeros(len(Q))
A = np.zeros((len(Q), len(Q)))
k = 0.1
B = np.zeros((len(Q), len(V)))

for sentence in training:
    for i in range(len(sentence)):
        # calculate pi matrix
        if i == 0:
            pi[Q[sentence[0][1]]] += 1
            word, tag = sentence[0]
            B[Q[tag]][V[word]] += 1
        else:
            pi[Q[sentence[i][1]]] += 1

            prev_tag = sentence[i-1][1]
            curr_tag = sentence[i][1]
            A[Q[prev_tag]][Q[curr_tag]] += 1

            word, tag = sentence[i]
            B[Q[tag]][V[word]] += 1

pi = pi / np.sum(pi)
A = A / np.sum(A, axis=1)[:, None]
B = (B + k) / np.sum(B + k, axis=1)[:, None]    


#Expected output:
#pi matrix such that pi[i] is the probability of starting in state q_i
#A matrix (Q x Q sized) such that A[i][j] is the probability of moving from state q_i to state q_j
#B matrix (Q x O sized) such that B[i][j] is the probability of state q_i to emit the word (observation) o_j


We have now a model and we can estimate its performance over the test set.

To do this, we need the Viterbi algorithm for the decoding. To help you, an implementation of Viterbi is provided:
(note: to use this version you need to assign a numeric id to each word and tag, if you haven't already)

In [114]:
"""
params is a triple (pi, A, B) where
pi = initial probability distribution over states
A = transition probability matrix
B = emission probability matrix

observations is the sequence of observations (in our case, the observed words)

the function returns the optimal sequence of states and its score
"""
def viterbi(params, observations):
    pi, A, B = params
    M = len(observations)
    S = pi.shape[0]

    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf') #cases that have not been treated
    backpointers = np.zeros((M, S), 'int')

    # base case
    alpha[0, :] = pi * B[:,observations[0]]

    # recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in range(S):
                score = alpha[t-1, s1] * A[s1, s2] * B[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    # now follow backpointers to resolve the state sequence
    ss = []
    ss.append(np.argmax(alpha[M-1,:]))
    for i in range(M-1, 0, -1):
        ss.append(backpointers[i, ss[-1]])

    return list(reversed(ss)), np.max(alpha[M-1,:])

#Example:

#original sentence: you can't very well sidle up to people on the street and ask if they want to buy a hot Bodhisattva .
#sentence as sequence of word indexes:
word_index= [953, 1856, 598, 115, 56043, 462, 58, 1069, 109, 30, 3702, 41, 713, 1068, 209, 2434, 58, 6953, 69, 755, 56044, 24]
predicted, score = viterbi((pi, A, B), word_index)
print(predicted)
#predicted will be a sequence of tag indexes:
#[12, 55, 86, 39, 29, 4, 70, 7, 14, 7, 0, 6, 21, 28, 12, 55, 28, 27, 28, 0, 9, 14, 15]

[55, 86, 39, 29, 4, 70, 7, 14, 7, 0, 6, 21, 28, 12, 55, 28, 27, 28, 0, 9, 15, 15]


In [115]:
#Example of results calculation
word_to_index = V
Q_ = list(Q.keys())
testing_formated = []
for sentence in testing:
    sent = ""
    words_index = [] #vector of word indices to be passed to Viterbi
    true_label= [] #vector of the true labels from labeled corpus
    for word,tag in sentence:
        words_index.append(word_to_index[word]) #word_to_index is a dictionary mapping a word to its index
        true_label.append(tag)
        sent=sent+" "+word
    testing_formated.append((words_index,true_label,sent))


for word_index,labels,sentence in testing_formated:
    print("  The sentence is:",sentence)
    print("   ##TRUE##    ##PRED##")
    predicted, score = viterbi((pi, A, B), word_index) #call the viterbi decoder
    for i,true_label in enumerate(labels):
        predicted_label = Q_[predicted[i]] #Q here is the vector of tags, so that at Q[i] we have the i_th tag in literal form
        print("      "+true_label+"         "+predicted_label)


  The sentence is:  you can't very well sidle up to people on the street and ask if they want to buy a hot Bodhisattva .
   ##TRUE##    ##PRED##
      PPSS         PPSS
      MD*         MD*
      QL         QL
      RB         RB
      VB         VBD
      IN         RP
      IN         IN
      NNS         NNS
      IN         IN
      AT         AT
      NN         NN
      CC         CC
      VB         VB
      CS         CS
      PPSS         PPSS
      VB         VB
      TO         TO
      VB         VB
      AT         AT
      JJ         JJ
      NP         .
      .         .
  The sentence is:  Additionally , since you're going to be hors de combat pretty soon with sprue , yaws , Delhi boil , the Granville wilt , liver fluke , bilharziasis , and a host of other complications of the hex you've aroused , you mustn't expect to be lionized socially .
   ##TRUE##    ##PRED##
      RB         RB
      ,         ,
      CS         CS
      PPSS+BER         PPSS+BER
      VBG     

**Exercise 3**: calculate Precision, Recall and F-measure for the bigram model

In [116]:
# Precison, Recall and F1-score for the bigram model
from sklearn.metrics import precision_recall_fscore_support
y_true = []
y_pred = []
for word_index,labels,sentence in testing_formated:
    predicted, score = viterbi((pi, A, B), word_index)
    for i,true_label in enumerate(labels):
        predicted_label = Q_[predicted[i]]
        y_true.append(true_label)
        y_pred.append(predicted_label)

precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)


Precision:  0.8617028156254097
Recall:  0.8744769874476988
F1-score:  0.8632846592775625


**Exercise 4**: modify your HMM to use trigrams instead of bigrams, and re-evaluate the results

## Using NLTK's HMM implementation

We will compare now our model built from scratch to the implementation provided by NLTK:

In [117]:
import nltk
from nltk.tag import hmm

trainer = hmm.HiddenMarkovModelTrainer(states = Q, symbols = V)

model = trainer.train_supervised(training, estimator=lambda fd, bins: hmm.LidstoneProbDist(fd, 0.1, bins))

for sent in testing:
    u_sent=[]
    for word, tag in sent:
        u_sent.append(word)
    tagged=model.tag(u_sent)
    print(sent)
    print(tagged)


[('you', 'PPSS'), ("can't", 'MD*'), ('very', 'QL'), ('well', 'RB'), ('sidle', 'VB'), ('up', 'IN'), ('to', 'IN'), ('people', 'NNS'), ('on', 'IN'), ('the', 'AT'), ('street', 'NN'), ('and', 'CC'), ('ask', 'VB'), ('if', 'CS'), ('they', 'PPSS'), ('want', 'VB'), ('to', 'TO'), ('buy', 'VB'), ('a', 'AT'), ('hot', 'JJ'), ('Bodhisattva', 'NP'), ('.', '.')]
[('you', 'PPSS'), ("can't", 'MD*'), ('very', 'QL'), ('well', 'RB'), ('sidle', 'VBD'), ('up', 'RP'), ('to', 'IN'), ('people', 'NNS'), ('on', 'IN'), ('the', 'AT'), ('street', 'NN'), ('and', 'CC'), ('ask', 'VB'), ('if', 'CS'), ('they', 'PPSS'), ('want', 'VB'), ('to', 'TO'), ('buy', 'VB'), ('a', 'AT'), ('hot', 'JJ'), ('Bodhisattva', '.'), ('.', '.')]
[('Additionally', 'RB'), (',', ','), ('since', 'CS'), ("you're", 'PPSS+BER'), ('going', 'VBG'), ('to', 'TO'), ('be', 'BE'), ('hors', 'FW-RB'), ('de', 'FW-IN'), ('combat', 'FW-NN'), ('pretty', 'QL'), ('soon', 'RB'), ('with', 'IN'), ('sprue', 'NN'), (',', ','), ('yaws', 'NNS'), (',', ','), ('Delhi', 'NP

**Exercise 5**: Calculate precision, recall and F-measure and compare them to the results that you obtained with the two models (bigram and trigram) that you implemented before. Can you deduce whether the NLTK model is using bigrams or trigrams? (It is not stated in the manual)

In [118]:
# Precison, Recall and F1-score for model trained with nltk
y_true = []
y_pred = []
for sent in testing:
    u_sent=[]
    for word, tag in sent:
        u_sent.append(word)
    tagged=model.tag(u_sent)
    for i, (word, tag) in enumerate(sent):
        y_true.append(tag)
        y_pred.append(tagged[i][1])

precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)

# The model trained with nltk has a 

Precision:  0.8611798030731084
Recall:  0.8702928870292888
F1-score:  0.8610531390544105


## Named Entity Recognition with Conditional Random Fields

For this exercise we will need to use the sklearn_crfsuite package. If it is not installed, it can be installed using pip with ```pip install sklearn-crfsuite```.

We will work on a Kaggle dataset named ```ner_dataset.csv``` (it should be in the same directory as the notebook).

Pandas can be used to read the content of the file:

In [119]:
import pandas as pd

data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill") #repeat sentence number on each row

words = list(set(data["Word"].values)) #vocabulary V
n_words = len(words)

print(words[:10])
print(n_words)

['Firdaus', 'Reggio', 'TSZ', 'doctrine', 'credit-card', 'Zinedine', '46,000', 'summed', 'U.S.-allied', 'misquoting']
35178


We provide you with some code that can read the sentences and produce the features in the format required by crf_suite. The ```SentenceGetter``` class transforms sentences into sequences of ```(word, POS, tag)``` triples

In [120]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
#load data
getter = SentenceGetter(data) #transform sentences into sequences of (Word, POS, Tag)
sentences = getter.sentences

The next function allows us to define features that are used in the CRF. The features are stored in a dictionary.

In [121]:
def word2features(sent, i):
    """
    input:
       sent: sentence in the format of sequence of (Word, POS, Tag) triples
       i: position in the sentence
    output:
       features: a dictionary mapping the feature name into a value
    """
    word = sent[i][0]
    postag = sent[i][1]

    features = { #features related to the current position
        'bias': 1.0,
        'word.lower()': word.lower(),
        'postag': postag,
    }
    if i > 0: #features related to preceding word/tag
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True #feature for Beginning of Sentence

    if i < len(sent)-1: #features related to the following word/tag
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True #feature for end of sentence

    return features


def sent2features(sent):
    #transforms the sentence in a sequence of features
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    #transforms the sentence in a sequence of labels
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    #transforms the sentence in a sequence of tokens (removes POS tags and labels)
    return [token for token, postag, label in sent]


We can now build the features and label vectors, and create a CRF model:

In [122]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

from sklearn_crfsuite import CRF
crf = CRF(algorithm='lbfgs',  max_iterations=100)


This will create a model with gradient descent algorithm ("lbfgs") and a limit of $100$ iterations.

Now we build the model and evaluate it on a 66/33 split between training and testing:

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
crf.fit(X_train, y_train)
pred=crf.predict(X_test)
n_pred = [item for sublist in pred for item in sublist]
n_test = [item for sublist in y_test for item in sublist]
report = classification_report(y_pred=n_pred, y_true=n_test)

print(report)

              precision    recall  f1-score   support

       B-art       0.40      0.01      0.03       137
       B-eve       0.63      0.30      0.40       111
       B-geo       0.84      0.90      0.87     12357
       B-gpe       0.98      0.83      0.90      5226
       B-nat       0.36      0.25      0.29        69
       B-org       0.77      0.69      0.73      6762
       B-per       0.83      0.79      0.81      5649
       B-tim       0.93      0.84      0.88      6650
       I-art       0.38      0.02      0.05       124
       I-eve       0.48      0.18      0.26        89
       I-geo       0.81      0.78      0.79      2433
       I-gpe       0.95      0.33      0.49        55
       I-nat       0.25      0.10      0.14        21
       I-org       0.74      0.80      0.76      5545
       I-per       0.84      0.89      0.86      5730
       I-tim       0.85      0.71      0.78      2110
           O       0.99      0.99      0.99    292571

    accuracy              

The report shows accuracy stats for all classes, but we are not interested in the **O** class. We can see that the scores for people names, place names and organizations are relatively low.

**Exercise 6**: Can you think of some new features for the CRF model to improve the results, especially on **B-org** ? Modify the *word2features* function to include the additional features and compare with the above results.

In [124]:
# modify the word2features function to include the word itself and other features
# There is improvement on B-org but there are some performance drops on other classes
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True

    return features

In [125]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

from sklearn_crfsuite import CRF
crf = CRF(algorithm='lbfgs',  max_iterations=100)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
crf.fit(X_train, y_train)
pred=crf.predict(X_test)
n_pred = [item for sublist in pred for item in sublist]
n_test = [item for sublist in y_test for item in sublist]
report = classification_report(y_pred=n_pred, y_true=n_test)

print(report)

              precision    recall  f1-score   support

       B-art       0.24      0.03      0.05       137
       B-eve       0.64      0.34      0.45       111
       B-geo       0.85      0.91      0.88     12357
       B-gpe       0.96      0.93      0.94      5226
       B-nat       0.60      0.13      0.21        69
       B-org       0.80      0.71      0.75      6762
       B-per       0.83      0.80      0.82      5649
       B-tim       0.92      0.87      0.89      6650
       I-art       0.23      0.04      0.07       124
       I-eve       0.53      0.22      0.31        89
       I-geo       0.82      0.81      0.81      2433
       I-gpe       0.93      0.49      0.64        55
       I-nat       0.00      0.00      0.00        21
       I-org       0.78      0.80      0.79      5545
       I-per       0.84      0.90      0.87      5730
       I-tim       0.82      0.76      0.79      2110
           O       0.99      0.99      0.99    292571

    accuracy              

## NER using a LSTM model

In this final section we will see an example of a neural network model written in PyTorch that uses a LSTM-based architecture for Named Entity Recognition.

First of all, we will prepare the data to have all information coded numerically (words and tags) and the sentences padded to a max length, in order to have all sentences of the same size.


In [126]:
#first of all we need to retrieve the number of different tags (a.k.a categories or classes of the words)
tags = list(set(data["Tag"].values))
n_tags=len(tags)

vocab = {w: i + 1 for i, w in enumerate(words)} #map words into a number
tag_map = {t: i for i, t in enumerate(tags)} #map tags into a number

from keras.preprocessing.sequence import pad_sequences
max_len=75

X = [[vocab[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words) #pad with special token PAD, with ID=n_words
y = [[tag_map[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=-1) # -1 is associated to the PAD token


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

Now we will load the data and split them into training and test. We set batch size at 32.

In [ ]:
import torch
from NERDataset import NERDataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

ner_train = NERDataset(X_train, y_train)
ner_test = NERDataset(X_test, y_test)

trainloader = torch.utils.data.DataLoader(ner_train, batch_size=32, shuffle=True)

The LSTM model is defined here. We have an embedding that maps each word in a vector (embedding) of size 100, which is learnt from the dataset. The embedded sentence is fed to a LSTM layer of size 50. The output is transferred to a fully connected layer with *n_tags* output, one for each of the possible labels. The loss is a cross-entropy loss over all tokens (excluding the "pad" tokens).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

vocab_size=n_words+1
embedding_dim=100
lstm_hidden_dim=50

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #maps each token to an embedding_dim vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #the LSTM takens embedded sentence
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, batch_first=True)

        #fc layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, n_tags)
    
    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim
        
        #run the LSTM along the sentences of length batch_max_len. We discard the cell state as output
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim                
        
        #reshape the Variable so that each row contains one token
        s = s.reshape(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim
        
        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags
        
        return F.log_softmax(s, dim=1)   # dim: batch_size*batch_max_len x num_tags
    
def loss_fn(outputs, labels):
    #reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)  
    
    #discard 'PAD' tokens
    mask = (labels >= 0).float()
    
    #the number of tokens is the sum of elements in mask
    num_tokens = int(torch.sum(mask))
    
    #pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels]*mask

    #cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs)/num_tokens
    


In the following block we carry out the training over 5 epochs

In [ ]:
network = Net()
# Initialize optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

num_epochs=5
#Run the training loop for defined number of epochs
for epoch in range(0, num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    i=0
    for inputs, targets in trainloader:
        #print(inputs, targets)
        
        optimizer.zero_grad() # Zero the gradients
        outputs = network(inputs) # Perform forward pass
        loss = loss_fn(outputs, targets) # Compute loss
        loss.backward() # Backprop
        optimizer.step() # Optimization
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
        i+=1

**Exercise 7**: Evaluate the model over the test set (see the data preparation cells), compare the result with the previously seen CRF model

In [127]:
# evaluate the model over the test set
from sklearn.metrics import classification_report

test_loader = torch.utils.data.DataLoader(ner_test, batch_size=32, shuffle=True)

# Function to evaluate the model on the test set
def evaluate_model(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            targets = np.array(targets.view(-1).tolist())
            positive_indices = np.where(targets >= 0)
            predicted = np.array(predicted.view(-1).tolist())
            # Append predictions and true labels to the lists
            y_true.extend(targets[positive_indices].tolist())
            y_pred.extend(predicted[positive_indices].tolist())

    return y_true, y_pred

y_true, y_pred = evaluate_model(network, test_loader)

print(classification_report(y_true, y_pred, target_names=tags))

# The performance are worst than the lastest models

NameError: name 'network' is not defined

**Exercise 8**: Modify the model to use GloVe vectors as initial embeddings (hint: see the documentation of the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html">Embedding</a> layer in PyTorch, in particular the *from_pretrained* method), and compare the results to the base one. 

In [ ]:
import torch
import torch.nn as nn
from torchtext.data import Field, LabelField
from torchtext.vocab import GloVe

glove = GloVe(name='6B', dim=embedding_dim)

text_field = Field(lower=True, include_lengths=True, batch_first=True)
label_field = LabelField()

examples = list(vocab.keys()) 

text_field.build_vocab(examples, vectors=glove)
label_field.build_vocab(examples)


Network = Net()
Network.embedding = nn.Embedding.from_pretrained(text_field.vocab.vectors)

# Initialize optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

num_epochs=5
#Run the training loop for defined number of epochs
for epoch in range(0, num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    i=0
    for inputs, targets in trainloader:
        #print(inputs, targets)
        
        optimizer.zero_grad() # Zero the gradients
        outputs = network(inputs) # Perform forward pass
        loss = loss_fn(outputs, targets) # Compute loss
        loss.backward() # Backprop
        optimizer.step() # Optimization
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
        i+=1

y_true, y_pred = evaluate_model(Network, test_loader)

print(classification_report(y_true, y_pred, target_names=tags))